NOTE: 2 of the .csv files in this come from github user vaastav, who stores data from previous FPL seasons.
https://github.com/vaastav/Fantasy-Premier-League

In [55]:
import requests
from requests.exceptions import ConnectionError
import time
import numpy as np
import pandas as pd   
import matplotlib as mpl 
import jellyfish

In [56]:
url = "https://fantasy.premierleague.com/api/bootstrap-static/"
found = False
i = 0
while i < 30 and not found:
    try:
        r = requests.get(url)
        found = True
    except ConnectionError:
        time.sleep(1)
        i += 1
json = r.json()

In [57]:
stats_df = pd.DataFrame(json['elements'])

In [58]:
stats_df.keys()

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'influence_rank', 'influence_rank_type', 'creativity_rank',
       'creativity_rank_type', 'threat_rank'

In [59]:
stats_data = stats_df[['code', 'first_name', 'second_name', 'team', 'element_type', 'now_cost', 'value_season', 'total_points', 'points_per_game', 'minutes', 'goals_scored', 'assists', 'goals_conceded', 'clean_sheets', 'influence', 'creativity', 'threat', 'ict_index_rank_type']]

In [60]:
stats_data.head()

,code,first_name,second_name,team,element_type,now_cost,value_season,total_points,points_per_game,minutes,goals_scored,assists,goals_conceded,clean_sheets,influence,creativity,threat,ict_index_rank_type
0,58822,Cédric,Alves Soares,1,2,45,10.7,48,2.3,1481,1,1,27,3,318.4,327.1,111.0,62
1,80201,Bernd,Leno,1,1,45,2.2,10,2.5,360,0,0,9,1,85.0,0.0,0.0,24
2,84450,Granit,Xhaka,1,3,50,12.0,60,2.2,2327,1,2,38,6,395.4,522.9,262.0,62
3,153256,Mohamed,Elneny,1,3,45,6.0,27,1.9,801,0,2,17,0,163.2,131.5,49.0,129
4,156074,Rob,Holding,1,2,45,6.4,29,1.9,840,1,0,16,2,253.0,8.6,49.0,106


In [61]:
stats_data.sort_values("total_points", ascending=False)

,code,first_name,second_name,team,element_type,now_cost,value_season,total_points,points_per_game,minutes,goals_scored,assists,goals_conceded,clean_sheets,influence,creativity,threat,ict_index_rank_type
296,118748,Mohamed,Salah,12,3,130,20.4,265,7.6,2758,23,14,22,17,1241.0,875.9,2230.0,1
453,85971,Son,Heung-min,18,3,120,21.5,258,7.4,3009,23,10,30,18,1201.8,1051.5,1296.0,2
298,169187,Trent,Alexander-Arnold,12,2,75,27.7,208,6.5,2853,2,12,22,18,865.2,1449.1,515.0,1
492,178186,Jarrod,Bowen,19,3,85,24.2,206,5.7,2987,12,17,46,8,822.2,627.2,1223.0,7
322,121145,João,Cancelo,13,2,70,28.7,201,5.6,3227,1,11,25,19,796.0,786.9,851.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,56917,Steve,Cook,16,2,45,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,160
398,57647,Lyle,Taylor,16,4,45,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,42
399,58771,Jack,Colback,16,3,45,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,170
400,102836,Brice,Samba,16,1,45,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,47


In [62]:
pd.options.mode.chained_assignment = None
stats_data.drop(stats_data[stats_data.minutes <300].index, inplace=True)

In [63]:
stats_data

,code,first_name,second_name,team,element_type,now_cost,value_season,total_points,points_per_game,minutes,goals_scored,assists,goals_conceded,clean_sheets,influence,creativity,threat,ict_index_rank_type
0,58822,Cédric,Alves Soares,1,2,45,10.7,48,2.3,1481,1,1,27,3,318.4,327.1,111.0,62
1,80201,Bernd,Leno,1,1,45,2.2,10,2.5,360,0,0,9,1,85.0,0.0,0.0,24
2,84450,Granit,Xhaka,1,3,50,12.0,60,2.2,2327,1,2,38,6,395.4,522.9,262.0,62
3,153256,Mohamed,Elneny,1,3,45,6.0,27,1.9,801,0,2,17,0,163.2,131.5,49.0,129
4,156074,Rob,Holding,1,2,45,6.4,29,1.9,840,1,0,16,2,253.0,8.6,49.0,106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,449988,Fabio,Silva,20,4,55,5.5,30,1.4,562,0,1,7,0,21.0,48.3,262.0,33
517,510362,Toti António,Gomes,20,2,40,1.5,6,1.5,360,0,0,6,0,87.0,4.5,28.0,128
519,159533,Adama,Traoré Diarra,20,3,55,8.0,44,2.2,1069,1,1,10,5,192.0,271.1,300.0,96
523,19624,João Filipe Iria,Santos Moutinho,20,3,50,19.2,96,2.7,2970,2,1,40,11,527.8,660.9,144.0,46


In [64]:
del stats_data["value_season"]
del stats_data['ict_index_rank_type']

In [65]:
stats_for_2122_season = stats_data

stats_for_2122_season['full_name'] = stats_for_2122_season['first_name'] + " " + stats_for_2122_season['second_name']

del stats_for_2122_season['first_name']
del stats_for_2122_season['second_name']

# moving full name column to the front
cols = stats_for_2122_season.columns.tolist()
cols = cols[-1:] + cols[:-1]
stats_for_2122_season = stats_for_2122_season[cols]

stats_for_2122_season

,full_name,code,team,element_type,now_cost,total_points,points_per_game,minutes,goals_scored,assists,goals_conceded,clean_sheets,influence,creativity,threat
0,Cédric Alves Soares,58822,1,2,45,48,2.3,1481,1,1,27,3,318.4,327.1,111.0
1,Bernd Leno,80201,1,1,45,10,2.5,360,0,0,9,1,85.0,0.0,0.0
2,Granit Xhaka,84450,1,3,50,60,2.2,2327,1,2,38,6,395.4,522.9,262.0
3,Mohamed Elneny,153256,1,3,45,27,1.9,801,0,2,17,0,163.2,131.5,49.0
4,Rob Holding,156074,1,2,45,29,1.9,840,1,0,16,2,253.0,8.6,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,Fabio Silva,449988,20,4,55,30,1.4,562,0,1,7,0,21.0,48.3,262.0
517,Toti António Gomes,510362,20,2,40,6,1.5,360,0,0,6,0,87.0,4.5,28.0
519,Adama Traoré Diarra,159533,20,3,55,44,2.2,1069,1,1,10,5,192.0,271.1,300.0
523,João Filipe Iria Santos Moutinho,19624,20,3,50,96,2.7,2970,2,1,40,11,527.8,660.9,144.0


The stats for the 21/22 season are now finished, and ready to be merged with the stats from the previous season, which will be gathered below.

In [66]:
stats_data = pd.read_csv("20_21Season.csv")

In [67]:
stats_data

,first_name,second_name,goals_scored,assists,total_points,minutes,goals_conceded,creativity,influence,threat,bonus,bps,ict_index,clean_sheets,red_cards,yellow_cards,selected_by_percent,now_cost,element_type
0,Mesut,Özil,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0.5,67,MID
1,Sokratis,Papastathopoulos,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0.1,48,DEF
2,David,Luiz Moreira Marinho,1,0,41,1396,20,46.7,250.6,144.0,0,230,43.9,3,1,1,0.8,54,DEF
3,Pierre-Emerick,Aubameyang,10,3,131,2330,26,361.5,493.8,823.0,11,358,167.4,10,0,2,7.2,113,MID
4,Cédric,Soares,0,1,28,744,11,114.8,110.8,66.0,3,125,29.2,2,0,1,0.3,46,DEF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,Andreas,Söndergaard,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0.2,40,GK
709,Patrick,Cutrone,0,0,2,23,0,0.0,0.0,4.0,0,5,0.2,0,0,0,0.0,59,FWD
710,Nigel,Lonwijk,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0.0,39,DEF
711,Willian José,Da Silva,1,1,40,1106,13,165.9,89.6,256.0,3,131,50.9,5,0,0,0.2,68,FWD


In [68]:
del stats_data['bonus']
del stats_data['bps']
del stats_data['yellow_cards']
del stats_data['red_cards']
del stats_data['selected_by_percent']

In [69]:
pd.options.mode.chained_assignment = None
stats_data.drop(stats_data[stats_data.minutes < 300].index, inplace=True)

In [70]:
stats_data = stats_data.replace({"GK": 1, "DEF": 2, "MID": 3, "FWD": 4})

In [71]:
stats_data

,first_name,second_name,goals_scored,assists,total_points,minutes,goals_conceded,creativity,influence,threat,ict_index,clean_sheets,now_cost,element_type
2,David,Luiz Moreira Marinho,1,0,41,1396,20,46.7,250.6,144.0,43.9,3,54,2
3,Pierre-Emerick,Aubameyang,10,3,131,2330,26,361.5,493.8,823.0,167.4,10,113,3
4,Cédric,Soares,0,1,28,744,11,114.8,110.8,66.0,29.2,2,46,2
5,Alexandre,Lacazette,13,3,129,1916,21,307.4,602.4,797.0,170.4,7,84,4
7,Bernd,Leno,0,0,131,3131,37,0.0,702.2,2.0,70.3,11,50,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,Vitor,Ferreira,0,1,26,509,8,179.0,80.0,72.0,33.3,0,47,3
698,Ki-Jana,Hoever,0,0,13,573,11,59.7,100.4,26.0,18.7,0,44,2
699,Nélson,Cabral Semedo,1,3,109,2983,45,412.9,595.2,297.0,130.7,9,51,2
701,Rayan,Ait Nouri,1,1,54,1400,23,234.5,243.2,196.0,67.4,3,50,2


In [72]:
stats_for_2021_season = stats_data

We now have two dataframes of data for the previous 2 premier league seasons, containing the 12 variables per player than we will run an analysis of. We will now combine these two dataframes into a single dataframe, and analyse it to build our model.

In [73]:
%store -r underlyingStats2021
%store -r underlyingStats2122

In [74]:
stats_for_2021_season['full_name'] = stats_for_2021_season['first_name'] + " " + stats_for_2021_season['second_name']

In [75]:
del stats_for_2021_season['first_name']
del stats_for_2021_season['second_name']

In [76]:
# moving full name column to the front
cols = stats_for_2021_season.columns.tolist()
cols = cols[-1:] + cols[:-1]
stats_for_2021_season = stats_for_2021_season[cols]

In [77]:
# sort all dataframes alphabetically by name
stats_for_2021_season.sort_values(by='full_name', inplace=True)
stats_for_2122_season.sort_values(by='full_name', inplace=True)
underlyingStats2021.sort_values(by='player_name', inplace=True)
underlyingStats2122.sort_values(by='player_name', inplace=True)

The number of players in the FPL dataframe and the Understat dataframe are conflicting, so will now find the anomalies and remove them. This process will be done by hand.

In [78]:
stats_for_2021_season.to_csv("FPL2021.csv", encoding='utf-8', index=False)
stats_for_2122_season.to_csv("FPL2122.csv", encoding='utf-8', index=False)
underlyingStats2021.to_csv("Understat2021.csv", encoding='utf-8', index=False)
underlyingStats2122.to_csv("Understat2122.csv", encoding='utf-8', index=False)

In [80]:
stats_data = pd.read_csv("19_20Season.csv")
del stats_data['bonus']
del stats_data['bps']
del stats_data['yellow_cards']
del stats_data['red_cards']
del stats_data['selected_by_percent']
pd.options.mode.chained_assignment = None
stats_data.drop(stats_data[stats_data.minutes < 300].index, inplace=True)
stats_data = stats_data.replace({"GK": 1, "DEF": 2, "MID": 3, "FWD": 4})

stats_data['full_name'] = stats_data['first_name'] + " " + stats_data['second_name']
cols = stats_data.columns.tolist()
cols = cols[-1:] + cols[:-1]
stats_data = stats_data[cols]

del stats_data['first_name']
del stats_data['second_name']

stats_data

,full_name,goals_scored,assists,total_points,minutes,goals_conceded,creativity,influence,threat,ict_index,clean_sheets,now_cost
0,Shkodran Mustafi,0,2,43,1205,17,45.5,277.2,155.0,47.9,4,51
1,H�ctor Beller�n,1,0,44,1156,18,76.9,187.8,103.0,37.0,4,52
2,Sead Kolasinac,0,2,55,1694,26,182.5,269.6,81.0,53.0,4,52
3,Ainsley Maitland-Niles,0,2,41,1382,22,182.0,301.8,58.0,53.6,3,45
4,Sokratis Papastathopoulos,2,0,57,1696,25,36.8,436.2,110.0,58.5,4,48
...,...,...,...,...,...,...,...,...,...,...,...,...
647,R�ben Diogo da Silva Neves,2,2,97,3053,35,445.4,538.6,307.0,129.1,13,52
648,Jo�o Filipe Iria Santos Moutinho,1,7,113,3105,36,1200.1,712.0,122.0,204.0,11,53
650,Adama Traor�,4,9,130,2599,33,786.4,727.2,583.0,209.7,7,56
653,Leander Dendoncker,4,0,100,2790,34,160.6,529.2,282.0,96.8,11,44


In [82]:
stats_data.to_csv("FPL1920.csv", encoding='utf-8', index=False)

All of the dataframes will now be aggregated manually, and analysed in ML_model_design.ipynb

In [ ]:
#UniqueNames = master_stats.element_type.unique()
#
##create a data frame dictionary to store your data frames
#DataFrameDict = {elem : pd.DataFrame() for elem in UniqueNames}
#
#for key in DataFrameDict.keys():
#    DataFrameDict[key] = master_stats[:][master_stats.element_type == key]

DataFrameDict is now a dict of 4 different dataframes containing players grouped by their position. We can now analyse the data by position, and create seperate data models for each position.